Lo cargamos desde un pickle para no perder los tipos de datos de las columnas, puesto que con csv los perderiamos

In [2]:
import polars as pl
import pickle


with open("datos_cleaned.pkl", "rb") as file:
    df = pickle.load(file)

df


Quantity,InvoiceDate,UnitPrice,Country,TotalSales
i64,date,f64,str,f64
1,2010-12-20,1.28,"""United Kingdom""",1.28
24,2011-10-17,4.25,"""United Kingdom""",102.0
10,2011-08-12,1.65,"""United Kingdom""",16.5
25,2011-09-21,0.42,"""United Kingdom""",10.5
1,2010-12-01,2.51,"""United Kingdom""",2.51
…,…,…,…,…
1,2011-09-12,2.08,"""United Kingdom""",2.08
1,2011-07-25,5.95,"""United Kingdom""",5.95
2,2011-05-13,0.85,"""United Kingdom""",1.7


In [3]:
df.describe()

statistic,Quantity,InvoiceDate,UnitPrice,Country,TotalSales
str,f64,str,f64,str,f64
"""count""",442815.0,"""442815""",442815.0,"""442815""",442815.0
"""null_count""",0.0,"""0""",0.0,"""0""",0.0
"""mean""",6.33038,"""2011-07-05 19:17:40.866000""",2.388613,null,11.456598
"""std""",6.458147,null,1.630417,null,12.759842
"""min""",1.0,"""2010-12-01""",0.12,"""Australia""",0.12
"""25%""",1.0,"""2011-03-29""",1.25,null,3.29
"""50%""",4.0,"""2011-07-22""",1.95,null,7.5
"""75%""",10.0,"""2011-10-20""",3.29,null,15.3
"""max""",27.0,"""2011-12-09""",7.5,"""United Kingdom""",193.96


Me he dado cuenta que en las ventas totales el maximo es muy superior al tercer percentil, vamos a calcular el IQR para comprobar el limete superior

In [ ]:
def calc_iqr_from_column_name(name:str)-> None:
    Q1 = df[name].quantile(0.25)
    Q3 = df[name].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    print(f"\nIQR for {name}")
    print(Q1, Q3, IQR, lower_bound, upper_bound)

calc_iqr_from_column_name("TotalSales")

Ahora filtramos por el limite superior

In [8]:
df.filter(df["TotalSales"] > 33.3)

Quantity,InvoiceDate,UnitPrice,Country,TotalSales
i64,date,f64,str,f64
24,2011-10-17,4.25,"""United Kingdom""",102.0
12,2011-11-08,2.95,"""United Kingdom""",35.4
18,2011-09-14,2.55,"""United Kingdom""",45.9
10,2011-08-16,6.75,"""United Kingdom""",67.5
8,2010-12-01,4.65,"""United Kingdom""",37.2
…,…,…,…,…
8,2011-04-21,4.95,"""United Kingdom""",39.6
12,2011-02-24,3.95,"""United Kingdom""",47.4
12,2011-06-09,2.95,"""Denmark""",35.4


Nos encontramos con mas de 20000 filas con valores mas altos de lo normal, para decidir si eliminarlos o no, vamos a crear dos nuevas variables, una que nos siga el dia de la semana y otra que nos diga si es
festivo en ese pais, todo esto es porque quizas estos valores tan altos son normales en relacion con el dia.


IQR for TotalSales
3.29 15.299999999999999 12.009999999999998 -14.724999999999998 33.315


Las acciones ejecutadas sobre el dataset como proceso de transformación de datos, justificando las técnicas utilizadas y las decisiones tomadas.

Las acciones ejecutadas sobre el dataset para crear los conjuntos de entrenamiento, validación y test, justificando las decisiones tomadas.
El entrenamiento de, al menos, un modelo de regresión, explicando la implementación realizada y mostrando claramente el resultado obtenido.


el dia de la semana puede ser util para detectrar patrones semanales
detectar si es dia festivo